# QRNN Model Selection

This notebook performs a grid search for the best performing neural network configuration for
quantile regression. As basic structure for the neural network a feed forward network is used and
the following paramters are varied:

- Network depth: 1 to 4 layers
- Network width: 16, 32, 64, 128, 256, 512 neurons
- Activation functions: linear, Sigmoid, ReLU, atan


## ipyparallel Setup

In [4]:
import ipyparallel as ipp
c     = ipp.Client(profile='mpi')
lview = c.load_balanced_view()

In [5]:
%%px
%env KERAS_BACKEND=tensorflow
%env OMP_NUM_THREADS=1
import matplotlib; matplotlib.use("agg")
import numpy as np
import sys
sys.path.append("/home/simonpf/src/typhon/")
from typhon.retrieval.qrnn import QRNN

[stdout:0] 
env: KERAS_BACKEND=tensorflow
env: OMP_NUM_THREADS=1
[stdout:1] 
env: KERAS_BACKEND=tensorflow
env: OMP_NUM_THREADS=1
[stdout:2] 
env: KERAS_BACKEND=tensorflow
env: OMP_NUM_THREADS=1
[stdout:3] 
env: KERAS_BACKEND=tensorflow
env: OMP_NUM_THREADS=1
[stdout:4] 
env: KERAS_BACKEND=tensorflow
env: OMP_NUM_THREADS=1
[stdout:5] 
env: KERAS_BACKEND=tensorflow
env: OMP_NUM_THREADS=1
[stdout:6] 
env: KERAS_BACKEND=tensorflow
env: OMP_NUM_THREADS=1
[stdout:7] 
env: KERAS_BACKEND=tensorflow
env: OMP_NUM_THREADS=1
[stdout:8] 
env: KERAS_BACKEND=tensorflow
env: OMP_NUM_THREADS=1
[stdout:9] 
env: KERAS_BACKEND=tensorflow
env: OMP_NUM_THREADS=1
[stdout:10] 
env: KERAS_BACKEND=tensorflow
env: OMP_NUM_THREADS=1
[stdout:11] 
env: KERAS_BACKEND=tensorflow
env: OMP_NUM_THREADS=1
[stdout:12] 
env: KERAS_BACKEND=tensorflow
env: OMP_NUM_THREADS=1
[stdout:13] 
env: KERAS_BACKEND=tensorflow
env: OMP_NUM_THREADS=1
[stdout:14] 
env: KERAS_BACKEND=tensorflow
env: OMP_NUM_THREADS=1
[stdout:15] 
env: KE

[stderr:0] Using TensorFlow backend.
[stderr:1] Using TensorFlow backend.
[stderr:2] Using TensorFlow backend.
[stderr:3] Using TensorFlow backend.
[stderr:4] Using TensorFlow backend.
[stderr:5] Using TensorFlow backend.
[stderr:6] Using TensorFlow backend.
[stderr:7] Using TensorFlow backend.
[stderr:8] Using TensorFlow backend.
[stderr:9] Using TensorFlow backend.
[stderr:10] Using TensorFlow backend.
[stderr:11] Using TensorFlow backend.
[stderr:12] Using TensorFlow backend.
[stderr:13] Using TensorFlow backend.
[stderr:14] Using TensorFlow backend.
[stderr:15] Using TensorFlow backend.
[stderr:16] Using TensorFlow backend.
[stderr:17] Using TensorFlow backend.
[stderr:18] Using TensorFlow backend.
[stderr:19] Using TensorFlow backend.
[stderr:20] Using TensorFlow backend.
[stderr:21] Using TensorFlow backend.
[stderr:22] Using TensorFlow backend.
[stderr:23] Using TensorFlow backend.


## Model Setup

In [6]:
%%px
quantiles = np.array([0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95])
def create_model(depth, width, act_fn):
    qrnn = QRNN(5, quantiles, depth, width, act_fn)
    return qrnn

## Cross Validation

Cross validation is used to determine the expected values of the quantile loss for each estimated quantile as well as the CRPS score of the estimated posterior.

In [7]:
%%px
from typhon.retrieval.scores import mean_quantile_score
x_train = np.load("src/atms_simulations/data/x_train_5.npy")
y_train = np.load("src/atms_simulations/data/y_train_5.npy")

def score(y_pred, y_test):
    quantile_scores = mean_quantile_score(y_pred, y_test, quantiles,
                                          convergence_epochs = 1,
                                          learning_rate_minimum = 1e-4,
                                          maximum_epochs = 400)
    crps = QRNN.crps(y_pred, y_test, quantiles)
    return np.append(quantile_scores, crps)

In [8]:
def run_cross_validation(config):
    depth, width, act_fn = config
    qrnn = create_model(depth, width, act_fn)
    return qrnn.cross_validation(x_train, y_train, 1.0, n_folds = 10)

## Running the Calculations

In [36]:
depths = [6, 8, 10]
widths = [8, 16, 32, 64, 96, 128, 256, 512]
act_funcs = ["linear", "tanh", "sigmoid", "relu"]
act_funcs = ["relu"]
configs = [(d, w, f) for d in depths for w in widths for f in act_funcs]
async_results = lview.map_async(run_cross_validation, configs)

In [ ]:
results = []
for i,r in enumerate(async_results):
    print(configs[i])
    print("Result: " + str(r))
    results += [(configs[i], r)]

(6, 8, 'relu')
Result: (0.98188435579109201, 0.015167518710878368)
(6, 16, 'relu')
Result: (0.91563199266052242, 0.0053207143314315912)
(6, 32, 'relu')
Result: (0.90462827638053889, 0.0078360805431844475)
(6, 64, 'relu')
Result: (0.89890136842918411, 0.0057325606718215448)
(6, 96, 'relu')
Result: (0.89850282076263421, 0.0077331700836060701)
(6, 128, 'relu')
Result: (0.89762011726379387, 0.0067965099447727063)
(6, 256, 'relu')
Result: (0.8965844658699037, 0.0066490351489058505)


In [22]:
import numpy as np

data = np.zeros((5, 7, 4, 2))
act_indices = {"linear" : 0, "tanh" : 1, "sigmoid" : 2, "relu" : 3}
for ((n_l, n_n, act), (m, s)) in results:
    data[n_l, int(np.log2(n_n)) - 3, act_indices[act], :] = np.array([m, s])

In [33]:
np.save("data/model_selection_structure", data)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib_settings
f, axs = plt.subplots(1, 3, figsize = (15, 5))

x = 2.0 ** np.arange(3, 10)

ax = axs[0]
for i in range(4):
    m = data[i + 1, :, 1, 0]
    s = data[i + 1, :, 1, 1]
    ax.plot(x, m, lw = 2)
    ax.fill_between(x, m + s, m - s, alpha = 0.2)
    ax.set_ylim([0.88, 1.1])
    ax.set_xscale("log")
    
ax = axs[1]
for i in range(4):
    m = data[i + 1, :, 2, 0]
    s = data[i + 1, :, 2, 1]
    ax.plot(x, m, lw = 2)
    ax.fill_between(x, m + s, m - s, alpha = 0.2)
    ax.set_ylim([0.88, 1.1])
    ax.set_xscale("log")
    
ax = axs[2]
for i in range(4):
    m = data[i + 1, :, 3, 0]
    s = data[i + 1, :, 3, 1]
    ax.plot(x, m, lw = 2)
    ax.fill_between(x, m + s, m - s, alpha = 0.2)
    ax.set_ylim([0.88, 1.1])
    ax.set_xscale("log")

In [16]:
import numpy as np
res = {"linear"  : np.zeros((len(depths), len(widths), 2)),
       "relu"    : np.zeros((len(depths), len(widths), 2)),
       "tanh"    : np.zeros((len(depths), len(widths), 2)),
       "sigmoid" : np.zeros((len(depths), len(widths), 2))}
inds = dict(zip(widths, range(len(widths))))

for ((n_layers, width, act), (mean, std)) in results:
    res[act][int(n_layers), inds[width], 0] = mean
    res[act][int(n_layers), inds[width], 1] = std

In [44]:
def print_table(res, fn = None):
    s = r""
    for j in range(res.shape[1]):
        s += r" & $n_n = {0}$ ".format(widths[j])
    s += r"\\ \hline"
    for i in range(res.shape[0]):
        s += "$n_h =  {0}$ & ".format(i)
        for j in range(res.shape[1] - 1):
            s += r"${:.2} \pm {:.2}$ & ".format(res[i, j, 0], res[i, j, 1])
        s += r"${:.2} \pm {:.2}$ \\ ".format(res[i, j, 0], res[i, j, 1])
    s+="\hline"
        
    if fn:
        f = open(fn, "w")
        f.write(s)
        f.close()
    else:
        return s    
    
def print_table2(res1, res2, fn = None):
    s = ""
    for i in range(res1.shape[0]):
        s += "$n_h = i$ &"
        for j in range(res1.shape[1]):
            s += r"${:.2} \pm {:.2}$ & ".format(res1[i, j, 0], res1[i, j, 1])
        for j in range(res2.shape[1] - 1):
            s += r"${:.2} \pm {:.2}$ & ".format(res2[i, j, 0], res2[i, j, 1])
        s += r"${:.2} \pm {:.2}$ \\ ".format(res2[i, j, 0], res2[i, j, 1])
        
    if fn:
        f = open(fn, "w")
        f.write(s)
        f.close()
    else:
        return s    

In [45]:
print_table(res["linear"],  "tables/linear.tbl")
print_table(res["sigmoid"], "tables/sigmoid.tbl")
print_table(res["tanh"],    "tables/tanh.tbl")
print_table(res["relu"],    "tables/relu.tbl")

## Training Parameters

Below the effect of training parameters on the QRNN performance is analyzed. This is again done by performing 10-fold cross-validation with varying training parameters. To save computation time the parameters are varied only independently. The following parameters are investigated:

- batch size: 128, 256, 512, 1024
- learning rate decay: 1.5, 2.0, 5.0, 10.0
- learning rate minimum: $10^{-4},10^{-5}, 10^{-6}, 10^{-7}, 10^{-8}$
- convergence epochs: 1, 2, 4, 8

In [5]:
def run_cross_validation(config):
    batch_size, lr_decay, lr_minimum, convergence_epochs = config
    qrnn = create_model(3, 128, "relu")
    return qrnn.cross_validation(x_train, y_train, 1.0, n_folds = 10,
                                 batch_size = batch_size,
                                 learning_rate_decay = lr_decay,
                                 learning_rate_minimum = lr_minimum,
                                 convergence_epochs = convergence_epochs)

In [6]:
configs = []
configs += [(bs, 2.0, 1e-6, 2) for bs in [128, 256, 512, 1024]]
configs += [(256, lrd, 1e-6, 2) for lrd in [1.5, 2.0, 5.0, 10.0]]
configs += [(256, 2.0, 10 ** -lrm, 2) for lrm in [4, 5, 6, 7, 8]]
configs += [(256, 2.0, 1e-6, ce) for ce in [1, 2, 4, 8]]

In [8]:
async_results = lview.map_async(run_cross_validation, configs)

In [9]:
results = []
for i,r in enumerate(async_results):
    print(configs[i])
    print("Result: " + str(r))
    results += [(configs[i], r)]

(128, 2.0, 1e-06, 2)
Result: (0.90578844951438897, 0.0081078108798440015)
(256, 2.0, 1e-06, 2)
Result: (0.91163818877601632, 0.0064749937914909033)
(512, 2.0, 1e-06, 2)
Result: (0.92271153948783868, 0.020830085622155849)
(1024, 2.0, 1e-06, 2)
Result: (0.93683845387268061, 0.020763810960643149)
(256, 1.5, 1e-06, 2)
Result: (0.91013961412239086, 0.0063265646771658041)
(256, 2.0, 1e-06, 2)
Result: (0.91320789633178712, 0.012580609311199118)
(256, 5.0, 1e-06, 2)
Result: (0.91662052987861631, 0.01052368406147664)
(256, 10.0, 1e-06, 2)
Result: (0.91795649431800841, 0.015360465991840955)
(256, 2.0, 0.0001, 2)
Result: (0.91043027124023435, 0.006420343418558328)
(256, 2.0, 1e-05, 2)
Result: (0.91255380694198607, 0.0070071830737801141)
(256, 2.0, 1e-06, 2)
Result: (0.91766805414772035, 0.010873377442336652)
(256, 2.0, 1e-07, 2)
Result: (0.91874591431236274, 0.014040805722100822)
(256, 2.0, 1e-08, 2)
Result: (0.91810450534629806, 0.021963042344801943)
(256, 2.0, 1e-06, 1)
Result: (0.9156301912040

In [12]:
configs = []
configs += [(64, lrd, 1e-6, 2) for lrd in [1.2, 1.5, 2.0]]
configs += [(64, 2.0, 10 ** -lrm, 2) for lrm in [3, 4, 5, 6]]
configs += [(64, 2.0, 1e-6, ce) for ce in [1, 2, 4, 8]]
async_results = lview.map_async(run_cross_validation, configs)

In [13]:
results = []
for i,r in enumerate(async_results):
    print(configs[i])
    print("Result: " + str(r))
    results += [(configs[i], r)]

(64, 1.2, 1e-06, 2)
Result: (0.89952332299423199, 0.0065377489453016082)
(64, 1.5, 1e-06, 2)
Result: (0.90161739125633233, 0.0086176645337436245)
(64, 2.0, 1e-06, 2)
Result: (0.90152903881454471, 0.00615886249641244)
(64, 2.0, 0.001, 2)
Result: (0.91008012251090997, 0.0050518619193384105)
(64, 2.0, 0.0001, 2)
Result: (0.90170960564231883, 0.0087728255696332968)
(64, 2.0, 1e-05, 2)
Result: (0.90167167879104626, 0.0068331846874113526)
(64, 2.0, 1e-06, 2)
Result: (0.900728275522232, 0.0093753006703030176)
(64, 2.0, 1e-06, 1)
Result: (0.90539736339759824, 0.0086392432600369915)
(64, 2.0, 1e-06, 2)
Result: (0.90346704487419127, 0.0066362010197610699)
(64, 2.0, 1e-06, 4)
Result: (0.90078221081542953, 0.0075512973896067958)
(64, 2.0, 1e-06, 8)
Result: (0.8986799619350434, 0.0041661199044197043)
